# 2.3	PÉLDA: UTAZÓ ÜGYNÖK PROBLÉMA MEGOLDÁSA

Egy ügynök több várost akar meglátogatni úgy, hogy minden városba pontosan egyszer lép be, és a teljes útvonalhosszt minimalizálja.


In [1]:
import numpy as np
import random

# Paraméterek
NUM_CITIES = 5
NUM_ANTS = 10
ALPHA = 1.0  # Feromon hatása
BETA = 2.0   # Heurisztikus hatás (távolság)
EVAPORATION_RATE = 0.5
ITERATIONS = 100

# Távolságmátrix
distance_matrix = np.random.randint(10, 100, (NUM_CITIES, NUM_CITIES))
np.fill_diagonal(distance_matrix, 0)  # Saját város távolsága 0

# Feromon mátrix
pheromone_matrix = np.ones_like(distance_matrix) * 0.1

# Fő algoritmus
for iteration in range(ITERATIONS):
    solutions = []  # Minden hangya útvonala és költsége

    for ant in range(NUM_ANTS):
        visited = [random.randint(0, NUM_CITIES - 1)]  # Kezdő város
        for step in range(NUM_CITIES - 1):
            current_city = visited[-1]
            probabilities = []

            for next_city in range(NUM_CITIES):
                if next_city not in visited:
                    pheromone = pheromone_matrix[current_city][next_city] ** ALPHA
                    heuristic = (1 / distance_matrix[current_city][next_city]) ** BETA
                    probabilities.append(pheromone * heuristic)
                else:
                    probabilities.append(0)

            # Normálás és város kiválasztása
            probabilities = np.array(probabilities) / np.sum(probabilities)
            next_city = np.random.choice(range(NUM_CITIES), p=probabilities)
            visited.append(next_city)

        # Út hosszának számítása
        cost = sum(distance_matrix[visited[i]][visited[i + 1]] for i in range(len(visited) - 1))
        solutions.append((visited, cost))

    # Feromon frissítés
    pheromone_matrix *= (1 - EVAPORATION_RATE)  # Párolgás
    for path, cost in solutions:
        for i in range(len(path) - 1):
            pheromone_matrix[path[i]][path[i + 1]] += 1 / cost  # Fordítottan arányos a költséggel

    # Legjobb megoldás iterációnként
    best_solution = min(solutions, key=lambda x: x[1])
    print(f"Iteration {iteration + 1}: Best cost = {best_solution[1]}")

# Eredmény
print("Best solution found:", best_solution)


Iteration 1: Best cost = 88
Iteration 2: Best cost = 80
Iteration 3: Best cost = 113
Iteration 4: Best cost = 80
Iteration 5: Best cost = 80
Iteration 6: Best cost = 88
Iteration 7: Best cost = 80
Iteration 8: Best cost = 80
Iteration 9: Best cost = 80
Iteration 10: Best cost = 88
Iteration 11: Best cost = 114
Iteration 12: Best cost = 80
Iteration 13: Best cost = 80
Iteration 14: Best cost = 114
Iteration 15: Best cost = 80
Iteration 16: Best cost = 80
Iteration 17: Best cost = 80
Iteration 18: Best cost = 80
Iteration 19: Best cost = 88
Iteration 20: Best cost = 80
Iteration 21: Best cost = 88
Iteration 22: Best cost = 80
Iteration 23: Best cost = 88
Iteration 24: Best cost = 80
Iteration 25: Best cost = 80
Iteration 26: Best cost = 80
Iteration 27: Best cost = 88
Iteration 28: Best cost = 80
Iteration 29: Best cost = 80
Iteration 30: Best cost = 114
Iteration 31: Best cost = 80
Iteration 32: Best cost = 80
Iteration 33: Best cost = 80
Iteration 34: Best cost = 80
Iteration 35: Best 

# MUNKAIDO-TERVEZES (ACO) SEGITSEGEVEL

## Feladatleírás:
Egy kisvállalat különböző munkafeladatokat kell kiosztania az alkalmazottak között úgy, hogy a munkaidő elosztása optimalizált legyen. A cél a következő:
1. Minimalizálni kell az alkalmazottak közötti munkaidő-különbséget (azaz egyenletes terhelés legyen).
2. Bizonyos feladatokat csak meghatározott alkalmazottak végezhetnek el. Erre érdemes létrehozni egy olyan task megoldási szerkezetet, ahol megmondjuk, hogy ki mit végezhet el. Taskokra lebontva.
3. Az alkalmazottak maximális napi munkaideje 8 óra.

Részletek:
* Feladatok: 10 feladat adott, melyek elvégzése különböző időtartamot igényel (órában).
  * tasks = [2, 4, 6, 1, 3, 5, 7, 3, 2, 4] # Feladatok időtartama (órákban)
* Alkalmazottak: 3 alkalmazott, akik különböző képességekkel rendelkeznek, így nem minden feladat végezhető el mindegyikük által.
  * employees = [0, 1, 2] # 3 alkalmazott
* Cél: Az ACO algoritmus segítségével minimalizálni kell a különbséget a legtöbb és legkevesebb munkaidővel rendelkező alkalmazott között.

Feladatlevezetés:
1. Adatok inicializálása:
  * Feladatok listája, időigényeikkel együtt.
  * Alkalmazottak listája, akikhez bizonyos feladatokat rendelhetünk.
  * Egy feromonmátrix, amely nyomon követi az alkalmazott-feladat párosok vonzerejét.
2. Átmeneti valószínűség kiszámítása:
  * Az egyes alkalmazottakhoz rendelhető feladatok valószínűségét az ACO szabályai szerint határozzuk meg:

\begin{align*}
P_{ij} = \frac{\tau_{ij}^\alpha \cdot \eta_{ij}^\beta}{\sum_{k \in S} \tau_{ik}^\alpha \cdot \eta_{ik}^\beta}
\end{align*}
ahol:
-  P_{ij}: Az i-ből j-be való átmenet valószínűsége.
-  \tau_{ij}∶ az aktuális feromonmennyiség.
-  \eta_{ij}∶  a heurisztikus információ, amely az adott alkalmazottnál maradt szabad munkaidőt veszi figyelembe.
-  α,β: Súlyozási paraméterek, amelyek a feromon és a heurisztikus érték relatív fontosságát szabályozzák.
3. Útvonalak generálása (megoldások előállítása):
  * Minden hangya (iteráció) megpróbál egy teljes munkabeosztást készíteni.
4. Értékelés (fitness):
  * Az útvonal minősége az alkalmazottak közötti munkaidő-különbségen alapul.
5. Feromonfrissítés:
  * A jó megoldásokra több feromon kerül, míg a kevésbé jók feromonjai párolognak.

Feladat levezetésének összegzése:
1. Kezdeti populáció generálása: Véletlenszerű munkarendeket állítunk elő.
2. Értékelés: Az egyes munkarendeket a munkaidők közötti különbség alapján értékeljük.
3. Módosítások (keresési lépések): Crossover és mutáció segítségével új munkarendeket állítunk elő.
4. Feromonfrissítés: Az eredmények alapján történik, bár itt a DEAP egyszerűbb GA implementációját használjuk.
5. Legjobb eredmény megjelenítése: A végső munkarend optimalizálja az alkalmazottak munkaidő-kiegyenlítését.


In [2]:
%pip install deap

Note: you may need to restart the kernel to use updated packages.


In [3]:
import random
from deap import base, creator, tools, algorithms

In [4]:
tasks = [2, 4, 6, 1, 3, 5, 7, 3, 2, 4]
# Feladatok időtartama (órákban)
employees = [0, 1, 2]
# 3 alkalmazott

In [5]:
# 1 el tudja végezni
# 0 nem tudja végezni
task_availability = [
    [1,1,1], # task 0
    [1,1,0], # task 1
    [0,1,1], # task 2
    [1,1,1], # task 3
    [0,1,1], # task 4
    [1,0,1], # task 5
    [1,1,0], # task 6
    [0,1,1], # task 7
    [1,1,0], # task 8
    [1,1,1], # task 9
]

In [6]:
def fitness_fg(time):
    workload = [0] * len(employees) 
    for task, employee in enumerate(time):
        workload[employee] += tasks[task]
    return (max(workload) - min(workload),)

In [7]:
def random_schedule():
    schedule = []
    for task in range(len(tasks)):
        valid_employees = [emp for emp, available in enumerate(task_availability[task]) if available]
        schedule.append(random.choice(valid_employees))
    return schedule

In [8]:
# GA initialization
creator.create("FitnessMin", base=base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [14]:
# Toolbox settings
toolbox = base.Toolbox()
toolbox.register("random_schedule", random_schedule)
toolbox.register("individual", tools.initIterate, creator.Individual,random_schedule)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [15]:
# Genetic operators
toolbox.register("evaluate", fitness_fg)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up= len(employees)-1, indpb= 0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

In [16]:
# Running the evolution
def run_evolution(pop_size=100, generations=50):
    population = toolbox.population(n=pop_size)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("avg", lambda x: sum(x)/len(x))
    stats.register("min", np.min)
    
    hof = tools.HallOfFame(1)
    population, log = algorithms.eaSimple(population, toolbox=toolbox, cxpb=0.7, mutpb=0.2, ngen=generations, 
                                          stats=stats, halloffame=hof, verbose=True)
    
    return population, log, hof

In [19]:
# main
if __name__ == "__main__":
    pop, log, hof = run_evolution()
    print("\nBest schedule: ", hof[0])
    
    workload = [0]*len(employees)
    for task, emp in enumerate(employees):
         workload[emp] += tasks[task]
    print("\nWorkload per employee: ", workload)
    print("\nFitness value", hof[0].fitness.values[0])

gen	nevals	avg  	min
0  	100   	11.21	1  
1  	75    	8.31 	1  
2  	78    	7.54 	1  
3  	69    	6.38 	1  
4  	68    	5.46 	1  
5  	66    	5.47 	1  
6  	73    	5.35 	1  
7  	81    	4.81 	1  
8  	71    	4.09 	1  
9  	80    	3.74 	1  
10 	75    	3.48 	1  
11 	78    	2.16 	1  
12 	83    	2.89 	1  
13 	74    	2.23 	1  
14 	78    	2.57 	1  
15 	72    	2.82 	1  
16 	86    	2.28 	1  
17 	75    	2.29 	1  
18 	86    	2.48 	1  
19 	76    	2.31 	1  
20 	69    	1.89 	1  
21 	72    	2.55 	1  
22 	84    	2.55 	1  
23 	79    	2.47 	1  
24 	85    	2.35 	1  
25 	82    	1.94 	1  
26 	76    	1.8  	1  
27 	82    	2.73 	1  
28 	72    	2.31 	1  
29 	62    	1.53 	1  
30 	73    	2.12 	1  
31 	68    	2.4  	1  
32 	75    	1.58 	1  
33 	78    	2.5  	1  
34 	70    	1.77 	1  
35 	71    	2.24 	1  
36 	75    	2.86 	1  
37 	87    	2.82 	1  
38 	66    	2.11 	1  
39 	86    	1.9  	1  
40 	72    	1.84 	1  
41 	76    	2.38 	1  
42 	70    	2.51 	1  
43 	70    	2.35 	1  
44 	77    	1.86 	1  
45 	66    	2.05 	1  
46 	86    	2.

# MUNKAIDO-TERVEZES (ACO) megoldás az utazó ügynök példája szerint

In [ ]:
import numpy as np
import random

tasks = [3, 4, 2, 5, 6] #(munkaórák).
task_availability = [[0, 1], [1, 2], [0, 2], [1, 2], [0, 1]]# (melyik feladatot ki végezheti el).

employees = [8, 8, 8] #(óra).

#ACO parameterei
ants = 10 #(hangya).
iteration = 50
evaporation_rate = 0.5
alpha = 1
beta = 2
q0 = 0.9

# inicializalas
num_tasks = len(tasks)
num_employees = len(employees)

# feromon matrix
pheromones = np.ones((num_tasks, num_employees))

# heurisztikus resz inicializalasa
heuristics = np.array([[1 if employee in task_availability[task] else 0 for employee in range(num_employees)] for task in range(num_tasks)])

def fittness_fg(schedule):
  workload = [0] * len(employees)
  for task, emp in enumerate(schedule):
    workload[emp] += tasks[task]
  fitness = max(workload) - min(workload)
  return fitness

# utvalasztas valoszinusege
def choose_task_employee(task, pheromones, heuristics):
  probabilities = np.zeros(num_employees)
  for employee in range(num_employees):
    if heuristics[task][employee] == 1:
      probabilities[employee] = pheromones[task][employee] ** alpha * heuristics[task][employee] ** beta

  prob_sum = np.sum(probabilities)
  probabilities /= prob_sum

  return np.random.choice(np.arange(num_employees), p=probabilities)

# hangya algoritmus
def ant_colony_optimization():
  global pheromones
  best_solution = None
  best_fitness = float('inf')

  for _ in range(iteration):
    solutions = []
    for _ in range(ants):
      solution = []
      for task in range(num_tasks):
        employee = choose_task_employee(task, pheromones, heuristics)
        solution.append(employee)

      fitness = fittness_fg(solution)
      solutions.append((solution, fitness))

      if fitness < best_fitness:
        best_fitness = fitness
        best_solution = solution
    # feromonok frissitese
    pheromones *= (1 - evaporation_rate)
    for solution, fitness in solutions:
      for task, employee in enumerate(solution):
        pheromones[task][employee] += 1.0 / (fitness + 1e-06)

  return best_solution, best_fitness

# main
if __name__ == '__main__':
  best_solution, best_fitness = ant_colony_optimization()
  print("\nLegjobb munkarend:", best_solution)
  workload = [0] * len(employees)
  for task, emp in enumerate(best_solution):
    workload[emp] += tasks[task]

  print("\nWorkload:", workload)


  print("Best fitness:", best_fitness)



Legjobb munkarend: [np.int64(1), np.int64(2), np.int64(2), np.int64(1), np.int64(0)]

Terheles alkalmazottaknal: [6, 8, 6]
Best fitness: 2
